<a href="https://colab.research.google.com/github/pallavibekal/IISC---Neural-Networks/blob/main/Hyper_Tuning_plus_kaggle_Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

To build and implement a Convolutional Neural Network model to classify between masked/unmasked/partially masked faces.

## Grading = 10 Points

In [1]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/MP2_FaceMask_Dataset.zip
!unzip -qq MP2_FaceMask_Dataset.zip
print("Data Downloaded Successfuly!!")

Data Downloaded Successfuly!!


In [5]:
#!wget -qq https://www.kaggle.com/c/facemask-detection/data
!unzip -qq /content/MP2_FaceMask_Dataset.zip

replace MP2_FaceMask_Dataset/train/with_mask/-110603108-gettyimages-533567012.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Import Required packages

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!unzip -qq /content/drive/MyDrive/FaceMask_Kaggle_test.zip

In [9]:
import numpy as np
import pandas as pd
import PIL
from matplotlib import pyplot as plt
import glob, os
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint

## Data Loading and preprocessing (2 points)

### Analyze the shape of images and distribution of classes

In [10]:
# Make a df for test and train

In [10]:
data_dir = "/content/MP2_FaceMask_Dataset/train"
test_data_dir = "/content/MP2_FaceMask_Dataset/test"

In [11]:
images_data = glob.glob("/content/FullIJCNN2013/*/*.ppm")
len(images_data), images_data[0]

IndexError: ignored

In [12]:
# Get file paths and labels
def extract_filenames (path):
  files = glob.glob(path + '/*')
  actions_list = []
  file_paths = []
  labels_list=[]
  for each in files:
    actions_list.append(each.split('/')[-1])
  for actions in actions_list:
    filepath = path + '/' + actions + '/*'
    filepaths = glob.glob(filepath)
    for x in filepaths:
      file_paths.append(x)
      labels_list.append(actions)
  return file_paths,labels_list

In [13]:
# Create a dataframe
def create_df(path):
  data_filepaths_list, labels = extract_filenames(path)
  df_ = pd.DataFrame(list(zip(data_filepaths_list, labels)),
                columns =['video_name', 'tag'])
  return df_

In [14]:
train_df = create_df(data_dir)
test_df = create_df(test_data_dir)

In [15]:
len(train_df['video_name']), len(test_df)

(5056, 1263)

In [37]:
test_df['tag'].value_counts()

without_mask    534
with_mask       406
partial_mask    323
Name: tag, dtype: int64

### Load the images using ImageDataGenerator

There are two main steps involved in creating the generator.
1. Instantiate ImageDataGenerator with required arguments to create an object
2. Use the `flow_from_directory` command depending on how your data is stored on disk. This is the command that will allow you to generate and get access to batches of data on the fly.

Hint: [link](https://keras.io/api/preprocessing/image/)

In [16]:
TRAINING_DIR = "/content/MP2_FaceMask_Dataset/train"
VALIDATION_DIR = "/content/MP2_FaceMask_Dataset/test/"
KAGGLE_TEST_DIR = '/content/FaceMask_Kaggle_test/FaceMask_Kaggle_test'

In [153]:
# Image Extraction using the ImageDataGenerator


train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   #featurewise_center=True,
                                   rotation_range=20,
                                   horizontal_flip=True,
                                   #vertical_flip=True,
                                   brightness_range=[0.4,1.5],
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   #validation_split=0.2,
                                   fill_mode='nearest',
                                   )

val_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                 )
                                 #horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale = 1./255)

batch_size = 10

training_set = train_datagen.flow_from_directory(TRAINING_DIR,
                                                 target_size = (224, 224),
                                                 interpolation="nearest",
                                                 class_mode='categorical',
                                                 classes=['partial_mask', 'with_mask', 'without_mask'],
                                                 #subset='training',
                                                 seed=11,
                                                 shuffle=True,
                                                 #color_mode="grayscale",
                                                 batch_size = 51
                                                 )
                                                 #zoom_range=0.2)

validation_set = val_datagen.flow_from_directory(VALIDATION_DIR,
                                                 target_size=(224, 224),
                                                 interpolation="nearest",
                                                 class_mode='categorical',
                                                 classes=['partial_mask', 'with_mask', 'without_mask'],
                                                 seed=11,
                                                 shuffle=True,
                                                 #color_mode="grayscale",
                                                 batch_size = 10)
                                                 #subset='validation')

#test_set = test_datagen.flow_from_directory(VALIDATION_DIR,
#                                            target_size = (224, 224),
                                            #batch_size = 1,
#                                            seed=11,
#                                            class_mode=None,
#                                            shuffle=True)
                                            #interpolation="nearest",
                                            #color_mode='grayscale',
#                                            class_mode='categorical',
#                                            classes=['partial_mask', 'with_mask', 'without_mask'])
                                            



Found 5029 images belonging to 3 classes.
Found 1259 images belonging to 3 classes.


In [91]:
next(training_set)[0].shape

(10, 224, 224, 3)

In [20]:
input_shap = next(training_set)[0].shape[1:]
input_shap

(224, 224, 3)

## Build the CNN model using Keras (4 points)



**Convolutional Neural Network:** A neural network in which at least one layer is a convolutional layer. A typical convolutional neural network consists of some combination of the following layers:

* convolutional layers
* pooling layers
* dense layers


**Conv2D**  

Passing an image with input shape of 3-D and to calculate the output: 

 $O = \frac{n - f + 2p}{s} + 1$

**MaxPool** 

The resulting output, when using the "valid" padding option, has a spatial shape (number of rows or columns) of: 

O = `math.floor`$(\frac{input shape - pool size)}{ strides}) + 1$ (when input shape >= pool size)

The resulting output shape when using the "same" padding option is: 

O = `math.floor`$(\frac{input shape - 1}{strides}) + 1$

by default, stride = None, so stride is same as pool size

Task-flow
* Initialize the network of convolution, maxpooling and dense layers
* Define the optimizer and loss functions
* Fit the model and evaluate the model

In [94]:
model = keras.Sequential([
#  keras.layers.Rescaling(1./255),
  keras.layers.Conv2D(256, kernel_size=(4,4), strides=(3,3), activation='relu',padding="same", input_shape=[224, 224, 3]),
  keras.layers.MaxPooling2D(pool_size=(2,2), strides=(1,1)),
  keras.layers.BatchNormalization(),
  #keras.layers.Conv2D(128, kernel_size=(7,7), strides=(3,3), activation='relu',padding="same"),
  #keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
  #keras.layers.BatchNormalization(),
  keras.layers.Conv2D(384, kernel_size=(5,5), strides=(2,2), activation='relu',padding="same"),
  keras.layers.MaxPooling2D(pool_size=(2,2), strides=(1,1)),
  keras.layers.BatchNormalization(),
  keras.layers.Conv2D(384, kernel_size=(3,3), strides=(2,2), activation='relu',padding="same"),
  keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
  keras.layers.BatchNormalization(),
  keras.layers.Dropout(0.5),
  keras.layers.Flatten(),
  #keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(192, activation='relu'),
  keras.layers.Dense(96, activation='relu'),
  keras.layers.Dropout(0.5),
  keras.layers.Flatten(),
  keras.layers.Dense(3, activation = 'softmax')
])


In [56]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 75, 75, 256)       4352      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 256)      0         
 2D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 74, 74, 256)      1024      
 hNormalization)                                                 
                                                                 
 conv2d_4 (Conv2D)           (None, 37, 37, 384)       2457984   
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 384)      0         
 2D)                                                             
                                                      

In [95]:
from tensorflow.keras.optimizers import SGD, Adam
optimizer = Adam(learning_rate=0.0001)

In [96]:
callbacks = [ keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss")]

In [97]:
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])


In [98]:
history = model.fit(training_set, validation_data=validation_set ,epochs = 25,callbacks=callbacks, class_weight={0:1.0,1:0.8,2:1.2}) #validation_data=test_ds)

Epoch 1/25
503/503 [==============================] - 112s 222ms/step - loss: 1.2303 - accuracy: 0.5826 - val_loss: 0.5211 - val_accuracy: 0.7998
Epoch 2/25
503/503 [==============================] - 110s 220ms/step - loss: 0.7945 - accuracy: 0.7021 - val_loss: 0.3674 - val_accuracy: 0.8562
Epoch 3/25
503/503 [==============================] - 110s 218ms/step - loss: 0.6341 - accuracy: 0.7566 - val_loss: 0.2936 - val_accuracy: 0.8999
Epoch 4/25
503/503 [==============================] - 110s 219ms/step - loss: 0.5715 - accuracy: 0.7841 - val_loss: 0.2909 - val_accuracy: 0.9031
Epoch 5/25
503/503 [==============================] - 110s 218ms/step - loss: 0.4743 - accuracy: 0.8127 - val_loss: 0.2496 - val_accuracy: 0.9317
Epoch 6/25
503/503 [==============================] - 110s 219ms/step - loss: 0.4581 - accuracy: 0.8200 - val_loss: 0.2565 - val_accuracy: 0.9325
Epoch 7/25
503/503 [==============================] - 110s 219ms/step - loss: 0.4037 - accuracy: 0.8485 - val_loss: 0.2161 -

In [92]:
model = keras.models.load_model('best_model.h5')

#test_loss, test_acc = model.evaluate(test_set)
#print("Test loss", test_loss)
#print("Test accuracy", test_acc)

## Transfer learning (4 points)

Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem.

A pre-trained model is a saved network that was previously trained on a large dataset, typically on a large-scale image-classification task.

The intuition behind transfer learning for image classification is that if a model is trained on a large and general enough dataset, this model will effectively serve as a generic model of the visual world. You can then take advantage of these learned feature maps without having to start from scratch by training a large model on a large dataset.

For eg. Using VGG16, we remove the last layer which takes a probability for each of the 1000 classes in the ImageNet and replaces it with a layer that takes 3 probabilities in our case.

### Use the pre-trained models ([VGG16](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16) or [ResNet50](https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/ResNet50))

* Load the pre-trained model
* Fit and evaluate the data

Hint: [How to use pre-trained model](https://towardsdatascience.com/step-by-step-guide-to-using-pretrained-models-in-keras-c9097b647b29)

#### Expected accuracy: More than 90%

Task-flow
* Initialize the network with the weights of Imagenet
* Fine tune the network by modifying fully connected layers.
* Re-train the model with our problem data

In [133]:
model_vgg = Sequential()
model_vgg.add(VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3)))
model_vgg.add(Flatten())
model_vgg.add(Dense(512,activation="relu",kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.02)))
model_vgg.add(BatchNormalization())
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(96,activation="relu",kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.02)))
model_vgg.add(BatchNormalization())
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(training_set.num_classes,activation='softmax'))
model_vgg.summary()



Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_18 (Flatten)        (None, 25088)             0         
                                                                 
 dense_35 (Dense)            (None, 512)               12845568  
                                                                 
 batch_normalization_14 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_36 (Dense)            (None, 96)                49248     
                                                     

In [144]:
for idx in range(len(model_vgg.layers)-1): #Leave the second to last layer trainable
  model.layers[idx].trainable = False
model_vgg.layers[len(model_vgg.layers)-1].trainable = True 

In [145]:
optimizer_adam = keras.optimizers.Adam(learning_rate=0.00001)
model_vgg.compile(optimizer=optimizer_adam, loss='categorical_crossentropy', metrics=['acc'])

In [119]:
from sklearn.utils import compute_class_weight

In [120]:
class_weight = compute_class_weight(class_weight='balanced',
                                        classes=np.unique(train_df['tag']),
                                        y=train_df['tag'])


In [130]:
class_weight

array([1.30242143, 1.03776683, 0.78827565])

In [146]:
callbacks = [ keras.callbacks.ModelCheckpoint("best_model_vgg1.h5", save_best_only=True, monitor="val_loss")]

In [154]:
history3 = model_vgg.fit(training_set,epochs=10,
                              validation_data=validation_set, class_weight={0:1.3,1:1,2:0.7},callbacks=callbacks)

Epoch 1/10
99/99 [==============================] - 113s 1s/step - loss: 0.3823 - acc: 0.9994 - val_loss: 0.3965 - val_acc: 0.9952
Epoch 2/10
99/99 [==============================] - 113s 1s/step - loss: 0.3713 - acc: 0.9982 - val_loss: 0.3752 - val_acc: 0.9968
Epoch 3/10
99/99 [==============================] - 113s 1s/step - loss: 0.3576 - acc: 0.9986 - val_loss: 0.3614 - val_acc: 0.9968
Epoch 4/10
99/99 [==============================] - 113s 1s/step - loss: 0.3458 - acc: 0.9992 - val_loss: 0.3780 - val_acc: 0.9921
Epoch 5/10
99/99 [==============================] - 112s 1s/step - loss: 0.3350 - acc: 0.9984 - val_loss: 0.3364 - val_acc: 0.9968
Epoch 6/10
99/99 [==============================] - 112s 1s/step - loss: 0.3229 - acc: 0.9984 - val_loss: 0.3182 - val_acc: 0.9984
Epoch 7/10
99/99 [==============================] - 111s 1s/step - loss: 0.3101 - acc: 0.9990 - val_loss: 0.3195 - val_acc: 0.9944
Epoch 8/10
99/99 [==============================] - 113s 1s/step - loss: 0.2977 - a

In [ ]:
df = pd.DataFrame()
df['img_path'] = glob.glob("FaceMask_Kaggle_test/*")
df['order'] = [int(i.split("/")[1][:-4]) for i in df['img_path']]
df.sort_values('order',inplace=True)
df.reset_index(inplace=True,drop=True)
df

In [99]:
kaggle_features = []
for i in df.img_path:

#  im = np.array(PIL.Image.open(i).convert('L').resize((224,224)))
  im = np.array(PIL.Image.open(i)  .resize((224,224)))
  if im.shape != (224,224,3):
    print(i, im.shape)
  kaggle_features.append(im)
  
kaggle_features = np.array(kaggle_features)

In [155]:
# Loading the saved model
model = keras.models.load_model('/content/best_model_vgg1.h5')

In [100]:
model = keras.models.load_model('/content/best_model.h5')

In [156]:
pred = model.predict(kaggle_features)

In [157]:
act_pred = np.argmax(pred,axis=1)
set(act_pred)

{0, 1, 2}

In [158]:
kaggle = pd.DataFrame()
kaggle['label'] = act_pred
kaggle['img_path'] = df['img_path']
kaggle['label'].replace(0,'partial_mask',inplace=True)
kaggle['label'].replace(1,'with_mask',inplace=True)
kaggle['label'].replace(2,'without_mask',inplace=True)

In [159]:
kaggle.to_csv("kaggle_submission_w13.csv",index=False)